In [35]:
from llama_index.core import VectorStoreIndex, Settings, SimpleDirectoryReader
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

In [10]:
documents = SimpleDirectoryReader("../data").load_data()
llm = Ollama(model="llama3", request_timeout=120.0)

In [3]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

c:\Users\Enes227\environments\llama_environment\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
vector_index = VectorStoreIndex.from_documents(documents,embed_model=Settings.embed_model,show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/22 [00:00<?, ?it/s]

In [39]:
query_engine = vector_index.as_query_engine(llm=llm)
response = query_engine.query(
    "What were the two main things the author worked on outside of school before college?"
)
print(response)

The two main things the author worked on outside of school before college were writing and programming. Specifically, he wrote short stories and tried to write programs on an IBM 1401 computer using early Fortran language.


In [6]:
import nest_asyncio
nest_asyncio.apply()


In [40]:
evaluator = FaithfulnessEvaluator(llm=llm)
eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))

True


In [41]:
response_str = response.response
for source_node in response.source_nodes:
    eval_result = evaluator.evaluate(
        response=response_str, contexts=[source_node.get_content()]
    )
    print(str(eval_result.passing))

True
False


In [42]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='3d43ebe7-ac8f-4279-ba10-9bfa55f06d58', embedding=None, metadata={'file_path': 'c:\\Yedek\\Dosyalar\\stuff\\ders\\NLP\\05 LlamaIndex\\LlamaIndex_Workspace\\Phase_2\\..\\data\\paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-05-29', 'last_modified_date': '2024-05-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='358bf59e-66c2-4810-bd20-b37224fa45d9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'c:\\Yedek\\Dosyalar\\stuff\\ders\\NLP\\05 LlamaIndex\\LlamaIndex_Workspace\\Phase_2\\..\\data\\paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain',

In [43]:
response.source_nodes[1]

NodeWithScore(node=TextNode(id_='b28f94d3-00d2-45eb-a089-0360b58a97c7', embedding=None, metadata={'file_path': 'c:\\Yedek\\Dosyalar\\stuff\\ders\\NLP\\05 LlamaIndex\\LlamaIndex_Workspace\\Phase_2\\..\\data\\paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-05-29', 'last_modified_date': '2024-05-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='358bf59e-66c2-4810-bd20-b37224fa45d9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'c:\\Yedek\\Dosyalar\\stuff\\ders\\NLP\\05 LlamaIndex\\LlamaIndex_Workspace\\Phase_2\\..\\data\\paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain',

In [45]:
query = "What were the two main things the author worked on outside of school before college?"
eval_result = evaluator.evaluate_response(query=query, response= response)
print(str(eval_result))

query='What were the two main things the author worked on outside of school before college?' contexts=['What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nTh

In [46]:
for source_node in response.source_nodes:
    eval_result = evaluator.evaluate(
        query=query,
        response=response_str,
        contexts=[source_node.get_content()],
    )
    print(str(eval_result.passing))

True
False


In [37]:
dataset_generator = RagDatasetGenerator.from_documents(
    documents=documents,
    llm=llm,
    num_questions_per_chunk=10,
)

rag_dataset = dataset_generator.generate_questions_from_nodes()
questions = [e.query for e in rag_dataset.examples]

In [38]:
questions

['Here are 10 questions based on the given text:',
 'What were the two main things the author worked on outside of school before college?',
 'What type of stories did the author write as a beginning writer?',
 "Where was the IBM 1401 computer located, and what was it used for in the author's junior high school?",
 'What programming language did the author use to write programs on the IBM 1401?',
 'How did the author learn that programs could not terminate, and what was the reaction of the data center manager?',
 "What kind of microcomputer did the author's friend build himself, and where did he get it from?",
 'Why did it take the author years to convince his father to buy a TRS-80 computer, and when did they eventually purchase one?',
 'What was the gold standard in microcomputers at the time, according to the author?',
 'What subject did the author initially plan to study in college, but ended up switching from after discovering it was not as exciting as expected?',
 'Here are 10 que